You can run this notebook directly on Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DaniAffCH/Vessel-Geometric-Transformers/blob/main/main.ipynb)

In [1]:
import sys

COLAB_RUNTIME = 'google.colab' in sys.modules
if COLAB_RUNTIME:
    !git clone https://github.com/DaniAffCH/Vessel-Geometric-Transformers.git
    !mv Vessel-Geometric-Transformers/* . 
    !pip install -q -r requirements.txt
else:
    !pip install -q -r requirements.txt
    !pre-commit autoupdate
    !pre-commit install



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
[https://github.com/psf/black] already up to date!
[https://github.com/pycqa/isort] already up to date!
[https://github.com/PyCQA/flake8] already up to date!
[https://github.com/pre-commit/mirrors-mypy] already up to date!
pre-commit installed at .git/hooks/pre-commit


Loading the configuration

In [2]:
from src.utils import load_config
import os
from config import DatasetConfig, TrainerConfig, BaselineConfig, GatrConfig

config_path = os.path.join("config","config.yaml")

config = load_config(config_path)
dataset_config: DatasetConfig = config.dataset
trainer_config: TrainerConfig = config.trainer
baseline_config: BaselineConfig = config.baseline
gatr_config: GatrConfig = config.gatr

Loading the dataset

In [3]:
from src.data import VesselDataModule


data = VesselDataModule(dataset_config)
print(f'Train size: {len(data.train_set)}')
print(f'Validation size: {len(data.val_set)}')
print(f'Test size: {len(data.test_set)}')
print(data.train_set[2])

/home/daniaffch/Desktop/Uni/Deep_Learning/Vessel-Geometric-Transformers/.venv/lib/python3.10/site-packages/torch_geometric/data/dataset.py:238: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experim

Train size: 2999
Validation size: 599
Test size: 401
Data(pos=[8397, 3], wss=[8397, 3], pressure=[8397], face=[3, 16790], inlet_index=[214], label=Category.Single)


# TODO: analisi dati preliminare. Classi bilanciate? Statistiche sul dataset? Qualche plot figo.

# TODO: equivariance check! 

Testing

In [9]:
from src.trainer import VesselTrainer
from src.models import Gatr

trainer = VesselTrainer(trainer_config)


model = Gatr(gatr_config)
trainer.fit(model, data)

epoch,▁▁▂▂▂▂▃▃▃▃▄▄▅▅▅▅▆▆▆▆▇▇▇▇██
test/acc,▁▁
test/f1,▁▁
test/loss,▁▁
train/acc,▁▆▇█████████
train/f1,▁▆▇█████████
train/loss,█▃▂▁▁▁▂▂▁▁▁▁
trainer/global_step,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▅▅▆▆▇▇▇▇████
val/acc,▂▁▇▇▇▇▇▇████
val/f1,▂▁▇▇▇▇▇▇████
val/loss,▆█▂▂▁▁▁▂▁▁▁▁


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/daniaffch/Desktop/Uni/Deep_Learning/Vessel-Geometric-Transformers/.venv/lib/python3.10/site-packages/lightning/pytorch/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/home/daniaffch/Desktop/Uni/Deep_Learning/Vessel-Geometric-Transformers/.venv/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:652: Checkpoint directory /home/daniaffch/Uni/Deep_Learning/Vessel-Geometric-Transformers/ckpt exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type              | Params | Mode 
--------------------------------------------------------------
0 | hsProjection    | EquiLinearLayer   | 576    | train
1 | backbone        | ModuleList        | 25.3 K | tr

/home/daniaffch/Desktop/Uni/Deep_Learning/Vessel-Geometric-Transformers/.venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/home/daniaffch/Desktop/Uni/Deep_Learning/Vessel-Geometric-Transformers/.venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Epoch 0: 100%|██████████| 750/750 [00:24<00:00, 30.33it/s, v_num=gpk9, val/loss=0.0296, val/acc=0.990, val/f1=0.990, train/loss=0.163, train/acc=0.964, train/f1=0.964]

Metric val/loss improved. New best score: 0.030
Epoch 0, global step 750: 'val/loss' reached 0.02961 (best 0.02961), saving model to '/home/daniaffch/Uni/Deep_Learning/Vessel-Geometric-Transformers/ckpt/epoch=0-step=750.ckpt' as top 1


...

Epoch 13: 100%|██████████| 750/750 [00:26<00:00, 28.66it/s, v_num=gpk9, val/loss=0.000644, val/acc=1.000, val/f1=1.000, train/loss=0.00276, train/acc=0.999, train/f1=0.999]

Epoch 13, global step 10500: 'val/loss' was not in top 1


Epoch 14: 100%|██████████| 750/750 [00:25<00:00, 29.17it/s, v_num=gpk9, val/loss=0.00746, val/acc=0.997, val/f1=0.997, train/loss=0.000274, train/acc=1.000, train/f1=1.000]

Monitored metric val/loss did not improve in the last 3 records. Best score: 0.000. Signaling Trainer to stop.
Epoch 14, global step 11250: 'val/loss' was not in top 1


Epoch 14: 100%|██████████| 750/750 [00:25<00:00, 29.17it/s, v_num=gpk9, val/loss=0.00746, val/acc=0.997, val/f1=0.997, train/loss=0.000274, train/acc=1.000, train/f1=1.000]


In [10]:
trainer.test(model, data)

Restoring states from the checkpoint path at /home/daniaffch/Uni/Deep_Learning/Vessel-Geometric-Transformers/ckpt/epoch=11-step=9000-v1.ckpt
/home/daniaffch/Desktop/Uni/Deep_Learning/Vessel-Geometric-Transformers/.venv/lib/python3.10/site-packages/lightning/fabric/utilities/cloud_io.py:57: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any 

Testing DataLoader 0: 100%|██████████| 101/101 [00:02<00:00, 47.12it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test/acc          │    0.9975062608718872     │
│          test/f1          │     0.997481107711792     │
│         test/loss         │   0.007669165730476379    │
└───────────────────────────┴───────────────────────────┘

Training Loop

In [6]:
from src.trainer import VesselTrainer
from src.models import BaselineTransformer

trainer = VesselTrainer(trainer_config)

model = BaselineTransformer(baseline_config)

trainer.fit(model, data)

epoch,▁▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
test/acc,▁
test/f1,▁
test/loss,▁
train/acc,▁▆▇▇███████████
train/f1,▁▆▇▇███████████
train/loss,█▂▂▁▁▁▁▁▁▁▁▁▂▁▁
trainer/global_step,▁▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
val/acc,▁▆▆▇▆▆▇▇█████▇█
val/f1,▁▆▆▇▆▆▇▇█████▇█
val/loss,█▃▂▂▂▂▁▁▁▁▁▁▁▁▁


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/daniaffch/Desktop/Uni/Deep_Learning/Vessel-Geometric-Transformers/.venv/lib/python3.10/site-packages/lightning/pytorch/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/home/daniaffch/Desktop/Uni/Deep_Learning/Vessel-Geometric-Transformers/.venv/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:652: Checkpoint directory /home/daniaffch/Uni/Deep_Learning/Vessel-Geometric-Transformers/ckpt exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type               | Params | Mode 
--------------------------------------------------------------
0 | encoder        | TransformerEncoder | 2.2 K  | train
1 | embedder       | Linear             | 272    | tr

/home/daniaffch/Desktop/Uni/Deep_Learning/Vessel-Geometric-Transformers/.venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/home/daniaffch/Desktop/Uni/Deep_Learning/Vessel-Geometric-Transformers/.venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Epoch 0: 100%|██████████| 750/750 [00:22<00:00, 32.63it/s, v_num=8vob, val/loss=0.049, val/acc=0.985, val/f1=0.984, train/loss=0.224, train/acc=0.912, train/f1=0.913]

Metric val/loss improved. New best score: 0.049
Epoch 0, global step 750: 'val/loss' reached 0.04897 (best 0.04897), saving model to '/home/daniaffch/Uni/Deep_Learning/Vessel-Geometric-Transformers/ckpt/epoch=0-step=750.ckpt' as top 1


...

Epoch 10: 100%|██████████| 750/750 [00:23<00:00, 31.52it/s, v_num=8vob, val/loss=5.18e-5, val/acc=1.000, val/f1=1.000, train/loss=8.12e-6, train/acc=1.000, train/f1=1.000]

Epoch 10, global step 8250: 'val/loss' was not in top 1


Epoch 11: 100%|██████████| 750/750 [00:24<00:00, 30.49it/s, v_num=8vob, val/loss=6e-5, val/acc=1.000, val/f1=1.000, train/loss=7.52e-6, train/acc=1.000, train/f1=1.000]   

Monitored metric val/loss did not improve in the last 3 records. Best score: 0.000. Signaling Trainer to stop.
Epoch 11, global step 9000: 'val/loss' was not in top 1


Epoch 11: 100%|██████████| 750/750 [00:24<00:00, 30.49it/s, v_num=8vob, val/loss=6e-5, val/acc=1.000, val/f1=1.000, train/loss=7.52e-6, train/acc=1.000, train/f1=1.000]


In [7]:
trainer.test(model, data)

Restoring states from the checkpoint path at /home/daniaffch/Uni/Deep_Learning/Vessel-Geometric-Transformers/ckpt/epoch=8-step=6750-v1.ckpt
/home/daniaffch/Desktop/Uni/Deep_Learning/Vessel-Geometric-Transformers/.venv/lib/python3.10/site-packages/lightning/fabric/utilities/cloud_io.py:57: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any u

Testing DataLoader 0: 100%|██████████| 101/101 [00:02<00:00, 41.89it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test/acc          │            1.0            │
│          test/f1          │            1.0            │
│         test/loss         │   1.246781994268531e-05   │
└───────────────────────────┴───────────────────────────┘

----